In [119]:
from typing import (
    Annotated,
    ClassVar,
    Generic,
    Literal,
    TypeAlias,
    TypeVar,
    Union,
    ParamSpec,
)
from typing import get_origin, get_args
from enum import Enum

def get_info(type_in):
    """
    Prints the origin and arguments of a type.
    """
    print(f"Type: {type_in}")
    print("Origin:", get_origin(type_in))
    print("Arguments:", get_args(type_in))
    print()


P = ParamSpec("P")
T = TypeVar("T")
MyAlias: TypeAlias = Generic[T]

class HttpMethod(str, Enum):
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"

# Test cases to print the results
get_info(Literal[42])
get_info(int)
get_info(ClassVar[int])
get_info(Generic)
get_info(Generic[T])
get_info(Union[T, int])
get_info(list[tuple[T, T]][int])
get_info(Annotated[str, "$"])
get_info(MyAlias)
get_info(HttpMethod)

Type: typing.Literal[42]
Origin: typing.Literal
Arguments: (42,)

Type: <class 'int'>
Origin: None
Arguments: ()

Type: typing.ClassVar[int]
Origin: typing.ClassVar
Arguments: (<class 'int'>,)

Type: <class 'typing.Generic'>
Origin: <class 'typing.Generic'>
Arguments: ()

Type: typing.Generic[~T]
Origin: <class 'typing.Generic'>
Arguments: (~T,)

Type: typing.Union[~T, int]
Origin: typing.Union
Arguments: (~T, <class 'int'>)

Type: list[tuple[int, int]]
Origin: <class 'list'>
Arguments: (tuple[int, int],)

Type: typing.Annotated[str, '$']
Origin: <class 'typing.Annotated'>
Arguments: (<class 'str'>, '$')

Type: typing.Generic[~T]
Origin: <class 'typing.Generic'>
Arguments: (~T,)

Type: <enum 'HttpMethod'>
Origin: None
Arguments: ()



In [266]:
from typing import (
    Annotated,
    Any,
    ClassVar,
    Generic,
    Literal,
    Optional,
    ParamSpec,
    TypeAlias,
    TypeVar,
    Union,
)
from typing import get_origin, get_args
from enum import Enum
from types import UnionType, NoneType
import inspect
import dataclasses as dc
from collections import OrderedDict


# Custom Typing
class UnsetType:
    """Utility for representing `Empty` values."""

    _instance: Optional["UnsetType"] = None

    # Type["UnsetType"]
    def __new__(cls: Any) -> "UnsetType":
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __bool__(self) -> bool:
        return False

    def __repr__(self) -> str:
        return "UNSET"

    def __str__(self) -> str:
        return ""


# Create a singleton instance
UNSET: UnsetType = UnsetType()


@dc.dataclass
class Field:
    annotation: Any
    origin: Any = None
    types: Any = None
    default: Any = UNSET
    required: bool = True
    is_union: bool = False
    is_choice: bool = False

    def __post_init__(self) -> None:
        self.origin = get_origin(self.annotation)
        self.types = get_args(self.annotation)
        # Update if origin found
        if self.origin:
            self.required = NoneType not in self.types
            self.is_union = self.origin in (UnionType, Union)
            self.is_choice = self.origin is Literal
        # Remove NoneType
        if not self.required:
            self.types = tuple(x for x in self.types if x is not NoneType)
        # Single-Type
        if len(self.types) > 1 and self.is_union:
            raise ValueError(
                f"Expected 1 type, but found {len(self.types)} types: {self.types}"
            )
        # Real-Type
        if len(self.types) == 1 and self.is_union:
            real = Field(self.types[0])
            if real.is_choice:
                self.is_choice = True
                self.types = real.types


@dc.dataclass
class Inspector:
    object: Any
    info: inspect.Signature | None = None
    parameters: OrderedDict | None = None

    def __post_init__(self) -> None:
        self.info = inspect.signature(self.object)


In [281]:
from typing import Literal, Union
from types import UnionType
import inspect

def demo(x: Literal["one", "two"] | None, y: Literal["one", "two"] | None = None):
    pass

signature = inspect.signature(demo)
params = signature.parameters
field = params.get("x")
default_value = field.default
if default_value is inspect.Parameter.empty:
    default_value = UNSET

field_info = Field(field.annotation)
field_info.default = default_value
field_info

Field(annotation=typing.Optional[typing.Literal['one', 'two']], origin=typing.Union, types=('one', 'two'), default=UNSET, required=False, is_union=True, is_choice=True)

In [278]:
class HttpEnum(str, Enum):
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"

class HttpNotEnum:
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"
